In [ ]:
import pandas as pd
import requests
import ipywidgets as widgets
import base64
from IPython.display import display, clear_output, HTML

INDEX_PAGE = "https://idr.openmicroscopy.org/webclient/?experimenter=-1"

TITLE = '<h1><em><b>IDR Phenotype Retrieving</b></em></h1>'
DESC = """\
<div class="app-sidebar">

<p>Given a gene symbol as input, the tool retrieves a list of correlated phenotypes from IDR database.</p>
</div>
"""

In [ ]:
class App:
    def __init__(self):
        self.table_container = widgets.Output()
        self.send_button, send_box = self.create_send_button()
        self.res_button = self.create_reset_button()
        self.container = widgets.VBox([send_box, widgets.HBox([self.send_button, self.res_button])])
        self.down_container = widgets.HBox([])
        self.final_container = widgets.VBox([
            widgets.HTML(TITLE, layout=widgets.Layout(margin='0 0 5em 0')),
            widgets.HTML(DESC, layout=widgets.Layout(margin='0 0 5em 0')),
            self.container,
            self.table_container,
            self.down_container
        ])
        
    def create_send_button(self):
        label = widgets.Label('Gene Symbol:')
        self.text_area = widgets.Text(placeholder='')
        send_b = widgets.Button(description='Submit', tooltip='submit your gene symbol', disabled=True)
        send_b.on_click(self.on_change_s)
        self.text_area.observe(self.on_change_text, names=['value'])
        sub_box = widgets.HBox([label,self.text_area])
        return(send_b, sub_box)
    
    def create_reset_button(self):
        reset_b = widgets.Button(description='Reset', tooltip='reset the environment to generate a new table', disabled=True)
        reset_b.on_click(self.on_click_r)
        return (reset_b) 
    
    def on_change_text(self,_):
        if (self.text_area.value == '') | (self.text_area.value.isspace()):
            self.send_button.disabled=True
        else:
            self.send_button.disabled=False
    
    def on_click_r(self,_):
        try:
            del(self.df)
            for i in self.down_container.children:
                remove = self.down_container.children[-1]
                self.down_container.children = self.down_container.children[:-1]
                remove.close()
        except NameError:
            print("")
        self.text_area.value=''
        self.res_button.disabled=True
        self.send_button.disabled=True
        self.table_container.clear_output(wait=False)
    
    def on_change_s(self,_):
        with self.table_container:
            self.table_container.clear_output(wait=False)
            loading = widgets.HTML(value='<i class=\"fa fa-spinner fa-spin fa-5x fa-fw\" style="color: #f09f14;"></i>')
            display(loading)
            self.send_button.disabled = True
            gene = self.text_area.value
            
            with requests.Session() as session:
                request = requests.Request('GET', INDEX_PAGE)
                prepped = session.prepare_request(request)
                response = session.send(prepped)
                if response.status_code != 200:
                    response.raise_for_status()

            KEY_VALUE_SEARCH = "https://idr.openmicroscopy.org/searchengine/api/v1/resources/{type}/search/?key={key}&value={value}"
            qs1 = {
                'type': 'image',
                'key': 'Gene Symbol',
                'value': gene}
            url = KEY_VALUE_SEARCH.format(**qs1)
            json = session.get(url).json()

            res = json["results"]["results"]
            names = []
            ids = []
            for i in res:
                try:
                    dic = pd.DataFrame(i["key_values"])
                    if not dic.loc[dic['name'] == "Phenotype Term Accession"].empty:
                        ids = ids + list(dic.loc[dic['name'] == "Phenotype Term Accession"]["value"].values)
                        names = names + list(dic.loc[dic['name'] == "Phenotype Term Name"]["value"].values)
                except IndexError:
                    continue

            self.df = pd.DataFrame(columns=["Pheno name", "Pheno id"])
            self.df["Pheno name"] = names
            self.df["Pheno id"] = ids
            self.df.drop_duplicates(inplace=True,ignore_index=True)
            self.table_container.clear_output(wait=False)
            
            if (len(self.df) == 0):
                print("No phenotypes found for gene " + gene)
                self.res_button.disabled = False
            else:
                print(str(len(self.df)) + " phenotypes found for " + gene + " gene:")
                display(HTML(self.df.to_html(justify="left", index=False)))
                self.csv_file = self.df.to_csv("result_table.csv",index=False)
                self.res_button.disabled = False
                htmlWidget = widgets.HTML(value="")
                self.create_download_link("result_table.csv", htmlWidget)
                self.down_container.children = tuple(list(self.down_container.children) + [htmlWidget])
            
    
    def create_download_link(self, filename, htmlWidget):  
        title="Click here to download the table in csv format"
        
        data = open(filename, "rb").read()
        b64 = base64.b64encode(data)
        payload = b64.decode()
        
        html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank"><button class="button-style">Download table as csv file</button></a>'
        htmlWidget.value = html.format(payload=payload,title=title,filename=filename)
    
    
app = App()    
app.final_container    
    